<a href="https://colab.research.google.com/github/vladimiralencar/DeepLearning-LANA/blob/master/LSTM/BRNN_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RNNs

A ideia básica de RNNs é fazer uso de informações de tipo sequencial na entrada. Essas redes são recorrentes porque executam os mesmos cálculos para todos os elementos de uma sequência de entradas e a saída de cada elemento depende, além da entrada atual, de todos os cálculos anteriores. As RNNs provaram ter um excelente desempenho em problemas como a previsão do próximo caracter em um texto ou, da mesma forma, a predição da próxima sequência de palavras em uma frase.

De fato, as RNNs executam a mesma computação em cada instância, em diferentes entradas da mesma sequência. Compartilhando os mesmos parâmetros, também, uma RNN reduz fortemente o número de parâmetros que a rede deve aprender durante a fase de treinamento, melhorando assim os tempos de treinamento.

Por exemplo, para calcular o gradiente (no tempo t = 4), é necessário propagar o gradiente calculado por três instantes do tempo anterior e depois somar os gradientes assim obtidos. De fato, uma sequência de entrada inteira normalmente é considerada um único elemento do conjunto de treinamento. Portanto, se o erro total é simplesmente a soma dos erros em cada instante de tempo (para cada elemento da sequência de entrada) como resultado, o gradiente de erro resulta ser a soma dos gradientes de erro em cada instante de tempo. Este procedimento é chamado de Backpropagation Through Time (BPTT).

No algoritmo de backpropagation, os pesos são ajustados em proporção ao erro de gradiente e para a forma como os gradientes são computados. Se os pesos forem pequenos, isso pode levar a uma situação chamada Vanishing Gradient, onde o sinal de gradiente é tão pequeno que o aprendizado torna-se muito lento ou deixa de funcionar completamente. Se os pesos nesta matriz forem grandes, isso pode levar a uma situação em que o sinal de gradiente é tão grande que pode levar a aprendizagem a divergir. Isso geralmente é referido como Exploding Gradient.

## Classificador de Imagens com LSTMs Bidirecionais

Embora as RNNs sejam amplamente utilizadas em tarefas de processamento de linguagem, elas podem ser usadas para resolver outros tipos de problemas, como por exemplo classificação de imagens. Neste exercício usaremos LSTMs Bidirecionais no TensorFlow. As RNNs bidirecionais são baseadas na ideia de que a saída no tempo t pode depender de elementos anteriores e futuros na sequência. Para atingir isso, a saída de dois modelos RNNs deve ser misturada - um executa o processo em uma direção e o segundo executa o processo na direção oposta. A rede divide os neurônios de uma RNN regular em duas direções, uma para direção de tempo positivo (forward state) e outra para direção de tempo negativo (backward state). Por esta estrutura, a camada de saída pode obter informações de estados passados e futuros.

Usaremos o dataset MNIST e para classificar as imagens usando uma Rede Neural Recorrente, devemos considerar cada linha de imagem como uma sequência de pixels, pois a forma da imagem MNIST é de 28 × 28 pixels, então vamos processar 28 sequências de 28 timesteps para cada amostra.

In [0]:
# Imports
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn

In [2]:
# Importando e carregando o dataset MNIST
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
# Hyperparâmetros de treinamento
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

In [0]:
# Parâmetros da rede
n_input = 28 
n_steps = 28 
n_hidden = 128 
n_classes = 10 

In [0]:
# Definindo x e y como Placeholders
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

In [0]:
# Inicializando pesos e bias
weights = {'out': tf.Variable(tf.random_normal([2*n_hidden, n_classes]))}
biases = {'out': tf.Variable(tf.random_normal([n_classes]))}

In [0]:
# Classe para o modelo
def BiRNN(x, weights, biases):
    x = tf.transpose(x, [1, 0, 2])
    x = tf.reshape(x, [-1, n_input])
    x = tf.split(axis=0, num_or_size_splits=n_steps, value=x)
    
    # Estamos implementando 2 modelos LSTM, um para forward e outro para backward
    lstm_fw_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    lstm_bw_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    
    # Bloco para capturar exceções
    try:
        # Executa a rede
        outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x, dtype=tf.float32)
    except Exception: 
        outputs = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [8]:
# Criando o modelo
modelo = BiRNN(x, weights, biases)

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [9]:
# Função de custo
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = modelo, labels = y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
# Otimizador
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [0]:
# Avaliação
correct_pred = tf.equal(tf.argmax(modelo,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [0]:
# Incialização das variáveis
init = tf.global_variables_initializer()

In [13]:
# Sessão TensorFlow
with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Acurácia em Treino = " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Otimização Finalizada!")

    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print("Acurácia em Teste:", \
sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

Iter 1280, Minibatch Loss= 1.880477, Acurácia em Treino = 0.38281
Iter 2560, Minibatch Loss= 1.616861, Acurácia em Treino = 0.44531
Iter 3840, Minibatch Loss= 1.343122, Acurácia em Treino = 0.60156
Iter 5120, Minibatch Loss= 1.125509, Acurácia em Treino = 0.64062
Iter 6400, Minibatch Loss= 0.926978, Acurácia em Treino = 0.67188
Iter 7680, Minibatch Loss= 0.770490, Acurácia em Treino = 0.73438
Iter 8960, Minibatch Loss= 0.662792, Acurácia em Treino = 0.78906
Iter 10240, Minibatch Loss= 0.562921, Acurácia em Treino = 0.82812
Iter 11520, Minibatch Loss= 0.562661, Acurácia em Treino = 0.82031
Iter 12800, Minibatch Loss= 0.484952, Acurácia em Treino = 0.87500
Iter 14080, Minibatch Loss= 0.541616, Acurácia em Treino = 0.82812
Iter 15360, Minibatch Loss= 0.575102, Acurácia em Treino = 0.80469
Iter 16640, Minibatch Loss= 0.372537, Acurácia em Treino = 0.90625
Iter 17920, Minibatch Loss= 0.428721, Acurácia em Treino = 0.82812
Iter 19200, Minibatch Loss= 0.402138, Acurácia em Treino = 0.88281
It